In [1]:
import sys,os
import pandas as pd
import random
import numpy as np

In [2]:
module_path=os.path.abspath('/root/NLP_warehouse/reconstruct/basictask/')
if module_path not in sys.path:
    sys.path.append(module_path)
print(sys.path)

['', '/root/miniconda3/envs/nlp_sr/lib/python36.zip', '/root/miniconda3/envs/nlp_sr/lib/python3.6', '/root/miniconda3/envs/nlp_sr/lib/python3.6/lib-dynload', '/root/.local/lib/python3.6/site-packages', '/root/miniconda3/envs/nlp_sr/lib/python3.6/site-packages', '/root/miniconda3/envs/nlp_sr/lib/python3.6/site-packages/IPython/extensions', '/root/.ipython', '/root/NLP_warehouse/reconstruct/basictask']


In [3]:
from tasks import cls
from evaluation import pairclsEvaluator
from readers.paircls import getExamples

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [4]:
data_folder='../../datasets/lcqmc'
label2id={"0":0,"1":1}
train_examples=getExamples(os.path.join(data_folder,'lcqmc_train.tsv'),label2id=label2id,filter_heads=True)

2021-08-17 10:43:13 - INFO - getExamples - 109 : Heads like : text_a	text_b	label

2021-08-17 10:43:14 - INFO - getExamples - 116 : *****************************Logging some train examples*****************************
2021-08-17 10:43:14 - INFO - getExamples - 117 : Total train nums is : 238766
2021-08-17 10:43:14 - INFO - getExamples - 120 : 什么牌子的米粉好？	什么牌子的米粉好	1
2021-08-17 10:43:14 - INFO - getExamples - 120 : 形容叫人忘了我的词语	我老做梦，梦到僵尸。怎么办	0
2021-08-17 10:43:14 - INFO - getExamples - 120 : 您可以点击查看一下。	亲您可以点击账户通然后删除一下银行卡就可以的	0
2021-08-17 10:43:14 - INFO - getExamples - 120 : 辽宁省北票市属于朝阳哪个区	辽宁省营口市欢心甸属于哪个区	0
2021-08-17 10:43:14 - INFO - getExamples - 120 : 部落冲突怎么刷宝石	唱吧刷金币怎么刷	0


In [5]:
dev_examples=getExamples(os.path.join(data_folder,'lcqmc_dev.tsv'),label2id=label2id,filter_heads=True,mode='dev')
test_examples=getExamples(os.path.join(data_folder,'lcqmc_test.tsv'),label2id=label2id,filter_heads=True,mode='test')

2021-08-17 10:43:14 - INFO - getExamples - 109 : Heads like : text_a	text_b	label

2021-08-17 10:43:14 - INFO - getExamples - 116 : *****************************Logging some dev examples*****************************
2021-08-17 10:43:14 - INFO - getExamples - 117 : Total dev nums is : 8802
2021-08-17 10:43:14 - INFO - getExamples - 120 : 网络的镁铝是什么意思	这个网络是什么意思	0
2021-08-17 10:43:14 - INFO - getExamples - 120 : 谁帮我注册个韩服跑跑卡丁车账号密码	谁能帮我注册韩服跑跑卡丁车啊	1
2021-08-17 10:43:14 - INFO - getExamples - 120 : 三桥东属于西安的哪个区？	上海有哪些区？哪个区最繁华？	0
2021-08-17 10:43:14 - INFO - getExamples - 120 : 形成日地距离适中的原因是什么?	日地距离是如何测出的？	0
2021-08-17 10:43:14 - INFO - getExamples - 120 : 爱情骗了你爱情骗了我是什么歌	我的微信上不去了，不知道怎么了	0
2021-08-17 10:43:14 - INFO - getExamples - 109 : Heads like : text_a	text_b	label

2021-08-17 10:43:14 - INFO - getExamples - 116 : *****************************Logging some test examples*****************************
2021-08-17 10:43:14 - INFO - getExamples - 117 : Total test nums is : 12500
2021-08-17 10:43:14 -

In [6]:
model_path='/data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/'
paircls_model=cls(model_path=model_path,label2id=label2id,is_finetune=False,device='cpu')

2021-08-17 10:43:14 - INFO - __init__ - 48 : The label2id is
 {"0": 0, "1": 1}
2021-08-17 10:43:14 - INFO - __init__ - 58 : Loading model from /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/, which is from huggingface model
2021-08-17 10:43:14 - INFO - get_config_dict - 177 : loading configuration file /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/config.json
2021-08-17 10:43:14 - INFO - from_pretrained - 404 : loading bert model file /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/
2021-08-17 10:43:14 - INFO - from_pretrained - 423 : BertConfig has been loaded from /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/config.json
2021-08-17 10:43:16 - INFO - from_pretrained - 124 : loading vocabulary file /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/vocab.txt


In [7]:
from torch.utils.data import DataLoader
batch_size=32
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

In [8]:
dev_sentences=[example.text_list for example in dev_examples]
dev_labels=[example.label for example in dev_examples]

In [9]:
print(dev_sentences[0],dev_labels[0])

['开初婚未育证明怎么弄？', '初婚未育情况证明怎么开？'] 1


In [10]:
evaluator=pairclsEvaluator(sentences_list=dev_sentences,labels=dev_labels,write_csv=False)

2021-08-17 10:43:16 - INFO - __init__ - 29 : Evalautor sentence like : 

2021-08-17 10:43:16 - INFO - __init__ - 31 : 开初婚未育证明怎么弄？	初婚未育情况证明怎么开？	1

2021-08-17 10:43:16 - INFO - __init__ - 31 : 谁知道她是网络美女吗？	爱情这杯酒谁喝都会醉是什么歌	0

2021-08-17 10:43:16 - INFO - __init__ - 31 : 人和畜生的区别是什么？	人与畜生的区别是什么！	1

2021-08-17 10:43:16 - INFO - __init__ - 31 : 男孩喝女孩的尿的故事	怎样才知道是生男孩还是女孩	0

2021-08-17 10:43:16 - INFO - __init__ - 31 : 这种图片是用什么软件制作的？	这种图片制作是用什么软件呢？	1



In [11]:
paircls_model.fit(is_pairs=True,train_dataloader=train_dataloader,evaluator=evaluator,epochs=5,
                  output_path='/data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/')

2021-08-17 10:43:16 - INFO - fit - 104 : 当前是双句子分类任务
2021-08-17 10:43:16 - INFO - fit - 112 : 一个epoch 下，每隔1492个step会输出一次loss，每隔3731个step会评估一次模型


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

2021-08-17 10:43:19 - INFO - __call__ - 47 : pairclsEvaluator: Evaluating the model on  dataset in epoch 0 after 1 steps:
2021-08-17 10:43:19 - INFO - predict - 223 : 当前是双句子分类任务预测


  0%|          | 0/276 [00:00<?, ?it/s]

2021-08-17 10:44:53 - INFO - __call__ - 54 : Accuracy: 0.493
2021-08-17 10:44:53 - INFO - __call__ - 61 : AUC: 0.491
2021-08-17 10:44:54 - INFO - save_pretrained - 509 : Model weights saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/pytorch_model.bin
2021-08-17 10:44:54 - INFO - save_pretrained - 150 : Configuration saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/config.json
2021-08-17 10:44:54 - INFO - save_vocab - 51 : Vocab saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/vocab.txt
2021-08-17 10:44:54 - INFO - fit - 187 : In epoch 0, training_step 0, the eval score is 0.490856635413655, previous eval score is -9999999, model has been saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/
2021-08-17 11:22:15 - INFO - fit - 165 : Epoch : 0, train_step : 1492/37310, loss_value : 0.3796851287468589 
2021-08-17 12:00:48 - INFO - fit - 165 : Epoch : 0, train_step : 2984/37310, loss_value : 0.23639671422601066 
2021-08-17 12:16:35 - INFO - __call__ - 47 : pai

  0%|          | 0/276 [00:00<?, ?it/s]

2021-08-17 12:17:47 - INFO - __call__ - 54 : Accuracy: 0.837
2021-08-17 12:17:47 - INFO - __call__ - 61 : AUC: 0.921
2021-08-17 12:17:48 - INFO - save_pretrained - 509 : Model weights saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/pytorch_model.bin
2021-08-17 12:17:48 - INFO - save_pretrained - 150 : Configuration saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/config.json
2021-08-17 12:17:48 - INFO - save_vocab - 51 : Vocab saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/vocab.txt
2021-08-17 12:17:48 - INFO - fit - 187 : In epoch 0, training_step 3731, the eval score is 0.920537952790054, previous eval score is 0.490856635413655, model has been saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/


KeyboardInterrupt: 